In [1]:
'''
First, create an inference script (here named inference.py) that loads the 
clustering model and returns cluster labels. Then, deploy the model as a SageMaker endpoint.
'''

'\nFirst, create an inference script (here named inference.py) that loads the clustering model and returns cluster labels. Then, deploy the model as a SageMaker endpoint.\n'

In [2]:
import boto3
import sagemaker
from sagemaker.model import Model
import os
import numpy as np
from inference import model_fn, input_fn, predict_fn, output_fn
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

bucket_name = "arxiv-project-bucket"
role = "arn:aws:iam::221082214706:role/MYLabRole"
region = "us-east-1"

sess = sagemaker.Session(boto_session=boto3.Session(region_name=region))
print("Using bucket:", bucket_name)

# Normally include S3 path for trained model here, shifted below for changes

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/20/25 20:30:02] INFO     Found credentials from IAM Role:                                   ]8;id=23110;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=18962;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[02/20/25 20:30:10] INFO     Found credentials from IAM Role:                                   ]8;id=157744;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=570155;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Using bucket: arxiv-project-bucket


In [3]:
# S3 path for the trained model from Notebook 3
model_s3_path = f"s3://{bucket_name}/models/model.tar.gz"

In [4]:
# SageMaker Model using the SKLearn container and our inference script
model = Model(
    model_data=model_s3_path,
    image_uri=sagemaker.image_uris.retrieve(
        framework="sklearn", region=region, version="0.23-1", py_version="py3"
    ),
    role=role,
    entry_point="inference.py",
    source_dir=".",  # Ensure all .py files are in directory
    sagemaker_session=sess,
)

# Deploy the model as an endpoint and return a predictor object
endpoint_name = "arxiv-clustering-endpoint"
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.4xlarge",
    endpoint_name=endpoint_name,
)

print("Endpoint deployed:", endpoint_name)

[02/20/25 20:30:11] INFO     Defaulting to only supported image scope: cpu.                       ]8;id=606481;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=223348;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#532\532]8;;\

                    INFO     Repacking model artifact (s3://arxiv-project-bucket/models/model.tar.gz), ]8;id=801441;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=763302;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py#819\819]8;;\
                             script artifact (.), and dependencies ([]) into single tar.gz file                    
                             located at                                                                            
                             s3://sagemaker-us-east-1-221082214706/sagemaker-scikit-learn-2025-02-20-2             
                             0-30-11-153/model.tar.gz. This may take some time depending on model                  
                             size...                                                                               

[02/20/25 20:32:55] INFO     Creating model with name:                                              ]8;id=234743;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=94544;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             sagemaker-scikit-learn-2025-02-20-20-32-55-916                                        

[02/20/25 20:32:56] INFO     Creating endpoint-config with name arxiv-clustering-endpoint           ]8;id=289379;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=938982;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

                    INFO     Creating endpoint with name arxiv-clustering-endpoint                  ]8;id=792705;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=349541;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

--------!Endpoint deployed: arxiv-clustering-endpoint


In [5]:
# Troubleshooting testing of Endpoint

In [6]:
sm_client = boto3.client('sagemaker', region_name=region)
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(response['EndpointStatus'])

[02/20/25 20:37:28] INFO     Found credentials from IAM Role:                                   ]8;id=533681;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=279420;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

InService


In [7]:
print("Predictor:", predictor)

Predictor: None


In [8]:
# Should not be none, deployment didnt return a predictor, must create manually

In [9]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=JSONSerializer(), # Can adjust to whatever type we want in and out
    deserializer=JSONDeserializer()
)

In [10]:
# Testiong endpoint
# Assumption of 10 feature input
sample_input = np.random.rand(1, 10).tolist()
print("Sending sample input:", sample_input)

Sending sample input: [[0.48849114789280523, 0.5724973710506618, 0.3977945955897776, 0.5475990069730806, 0.2547959814606805, 0.5824504596854302, 0.9678382025725436, 0.7098109378163856, 0.37755924551622144, 0.05396033725406124]]


In [11]:
# Invoke the endpoint and print the result
response = predictor.predict(sample_input)
print("Endpoint response:", response)

Endpoint response: {'prediction': [3]}


In [12]:
# Belongs to cluster 4. Uncomment below cells to delete endpoint

In [15]:
'''
# ONLY FOR DELETION
sagemaker_client = boto3.client('sagemaker', region_name=region)

# Delete the endpoint first
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
print(f"Endpoint '{endpoint_name}' deletion initiated.")
'''

Endpoint 'arxiv-clustering-endpoint' deletion initiated.


In [16]:
'''
# Optionally, delete the endpoint configuration too.
# Note: Often the endpoint configuration name is the same as the endpoint name,
# but if you provided a custom EndpointConfigName during deployment, use that instead.
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
print(f"Endpoint configuration '{endpoint_name}' deleted.")
'''

Endpoint configuration 'arxiv-clustering-endpoint' deleted.
